In [7]:
import  tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

(xs, ys),_ = datasets.mnist.load_data()
print('datasets:', xs.shape, ys.shape, xs.min(), xs.max())

xs = tf.convert_to_tensor(xs, dtype=tf.float32) / 255.  # still same shape
db = tf.data.Dataset.from_tensor_slices((xs,ys))
db = db.batch(32).repeat(10)
print('datasets:', db)

datasets: (60000, 28, 28) (60000,) 0 255
datasets: <RepeatDataset shapes: ((None, 28, 28), (None,)), types: (tf.float32, tf.uint8)>


In [8]:
network = Sequential([layers.Dense(256, activation='relu'),   # (784 + 1) * 256
                     layers.Dense(256, activation='relu'),   # 257 * 256
                     layers.Dense(256, activation='relu'),
                     layers.Dense(10)])
network.build(input_shape=(None, 28*28))
network.summary()

optimizer = optimizers.SGD(lr=0.01)
acc_meter = metrics.Accuracy()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  65792     
_________________________________________________________________
dense_2 (Dense)              multiple                  65792     
_________________________________________________________________
dense_3 (Dense)              multiple                  2570      
Total params: 335,114
Trainable params: 335,114
Non-trainable params: 0
_________________________________________________________________


In [9]:
# use GradientTape and Keras.Model.train()
for step, (x,y) in enumerate(db):

    with tf.GradientTape() as tape:
        # [b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # [b, 784] => [b, 10]
        out = network(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10)
        # [b, 10]
        loss = tf.square(out-y_onehot)
        # [b]
        loss = tf.reduce_sum(loss) / 32


    acc_meter.update_state(tf.argmax(out, axis=1), y)

    grads = tape.gradient(loss, network.trainable_variables)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))


    if step % 200==0:

        print(step, 'loss:', float(loss), 'acc:', acc_meter.result().numpy())
        acc_meter.reset_states()

0 loss: 1.1549545526504517 acc: 0.0625
200 loss: 0.43066316843032837 acc: 0.65515625
400 loss: 0.36594605445861816 acc: 0.8340625
600 loss: 0.3201281428337097 acc: 0.8645313
800 loss: 0.2777119576931 acc: 0.89234376
1000 loss: 0.30878007411956787 acc: 0.891875
1200 loss: 0.2879890203475952 acc: 0.9109375
1400 loss: 0.24802908301353455 acc: 0.9184375
1600 loss: 0.21240770816802979 acc: 0.909375
1800 loss: 0.17359581589698792 acc: 0.9323437
2000 loss: 0.2074495553970337 acc: 0.94109374
2200 loss: 0.13924705982208252 acc: 0.93296874
2400 loss: 0.2214827686548233 acc: 0.9315625
2600 loss: 0.19837629795074463 acc: 0.93828124
2800 loss: 0.14539489150047302 acc: 0.93703127
3000 loss: 0.21524688601493835 acc: 0.934375
3200 loss: 0.1776341199874878 acc: 0.9398438
3400 loss: 0.14224880933761597 acc: 0.9365625
3600 loss: 0.12950338423252106 acc: 0.938125
3800 loss: 0.17978161573410034 acc: 0.9575
4000 loss: 0.20065954327583313 acc: 0.953125
4200 loss: 0.157958984375 acc: 0.94671875
4400 loss: 0.1